In [1]:
import logging
import asyncio
import pandas as pd
from IPython.display import display, Markdown

from datetime import datetime

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.config.enumerations import Channels
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials

from tastytrade.messaging.processors import TelegrafHTTPEventProcessor, RedisEventProcessor
from tastytrade.config import RedisConfigManager
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.utils.time_series import forward_fill

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

2025-03-13 20:09:46 - INFO:root:62:Logging initialized - writing to ../logs/dev_tastytrade_20250313.log


# Service Connections

In [2]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")

dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

influx_user = InfluxCredentials(config=config)
influxdb = influxdb_client.InfluxDBClient(
    url=influx_user.url, token=influx_user.token, org=influx_user.org
)

for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())
    handler.add_processor(RedisEventProcessor())

2025-03-13 20:09:50 - INFO:tastytrade.config.manager:170:Initialized 16 variables from .env file in Redis
2025-03-13 20:09:50 - DEBUG:asyncio:848:Get address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32>
2025-03-13 20:09:50 - DEBUG:asyncio:858:Getting address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32> took 18.306ms: [(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('170.76.244.141', 443))]
2025-03-13 20:09:50 - DEBUG:asyncio:535:<asyncio.sslproto.SSLProtocol object at 0x751c9c256790> starts SSL handshake
2025-03-13 20:09:50 - DEBUG:asyncio:594:<asyncio.sslproto.SSLProtocol object at 0x751c9c256790>: SSL handshake took 40.9 ms
2025-03-13 20:09:50 - DEBUG:asyncio:1121:<asyncio.TransportSocket fd=73, family=2, type=1, proto=6, laddr=('172.18.0.7', 48484), raddr=('170.76.244.141', 443)> connected to None:None: (<asyncio.sslproto._SSLProtocolTransport object at 0

2025-03-13 20:09:51 - INFO:tastytrade.messaging.handlers:212:SETUP
2025-03-13 20:09:51 - INFO:tastytrade.messaging.handlers:215:AUTH_STATE:UNAUTHORIZED
2025-03-13 20:09:51 - INFO:tastytrade.messaging.handlers:215:AUTH_STATE:AUTHORIZED
2025-03-13 20:09:51 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:1
2025-03-13 20:09:51 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:3
2025-03-13 20:09:51 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:5
2025-03-13 20:09:51 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:7
2025-03-13 20:09:51 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:9
2025-03-13 20:09:51 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:11
2025-03-13 20:09:51 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:99
2025-03-13 20:09:51 - INFO:tastytrade.messaging.handlers:223:FEED_CONFIG:5:COMPACT
2025-03-13 20:09:51 - INFO:tastytrade.messaging.handlers:223:FEED_CONFIG:7:COMPACT
2025-03-13 20:09:51 - INFO:tastytrade.messaging.han

# Market Data Subscriptions

In [3]:
start_time = datetime(2025, 3, 12)

symbols = ["BTC/USD:CXTALP", "NVDA", "AAPL", "QQQ", "SPY", "SPX", "/ESH25:XCME"]
intervals = ["1d", "1h", "30m", "15m", "5m", "m"]

# ticker subscriptions
await dxlink.subscribe(symbols)

# candle subscriptions
for symbol in symbols:
    for interval in intervals:
        coroutine = dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )
        await asyncio.wait_for(coroutine, timeout=10)

2025-03-13 20:09:56 - INFO:tastytrade.connections.sockets:231:Added subscription: BTC/USD:CXTALP
2025-03-13 20:09:56 - INFO:tastytrade.connections.sockets:231:Added subscription: NVDA
2025-03-13 20:09:56 - INFO:tastytrade.connections.sockets:231:Added subscription: AAPL
2025-03-13 20:09:56 - INFO:tastytrade.connections.sockets:231:Added subscription: QQQ
2025-03-13 20:09:56 - INFO:tastytrade.connections.sockets:231:Added subscription: SPY
2025-03-13 20:09:56 - INFO:tastytrade.connections.sockets:231:Added subscription: SPX
2025-03-13 20:09:56 - INFO:tastytrade.connections.sockets:231:Added subscription: /ESH25:XCME
2025-03-13 20:09:56 - INFO:tastytrade.connections.sockets:231:Added subscription: BTC/USD:CXTALP{=d}
2025-03-13 20:09:56 - INFO:tastytrade.connections.sockets:231:Added subscription: BTC/USD:CXTALP{=h}
2025-03-13 20:09:56 - INFO:tastytrade.connections.sockets:231:Added subscription: BTC/USD:CXTALP{=30m}
2025-03-13 20:09:56 - INFO:tastytrade.connections.sockets:231:Added subs

In [4]:
# forward fill
for symbol in symbols:
    for interval in intervals:
        event_symbol = f"{symbol}{{={interval}}}"
        logging.debug("Forward-filling %s", event_symbol)
        forward_fill(symbol=event_symbol, lookback_days=5)

# Check | Market Data feeds

In [4]:
symbol = "SPX{=m}"
symbol = "BTC/USD:CXTALP{=5m}"
symbol = "/ESH25:XCME{=m}"

display(Markdown(f"**Candle Feed:** {symbol}"))
display(
    dxlink.router.handler[Channels.Candle]
    .processors["feed"]
    .frames[f"{symbol}"]
    .tail(5)
    .sort(by="time", descending=True)
)

**Candle Feed:** /ESH25:XCME{=m}

eventSymbol,time,eventFlags,index,sequence,count,open,high,low,close,volume,bidVolume,askVolume,openInterest,vwap,impVolatility
str,datetime[μs],i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""/ESH25:XCME{=m}""",2025-03-13 20:10:00,0,7481388930013593600,0,260,5540.0,5540.25,5538.75,5538.75,640.0,504.0,136.0,2.091047e6,5539.61,0.26
"""/ESH25:XCME{=m}""",2025-03-13 20:09:00,0,7481388672315555840,0,871,5538.25,5540.25,5538.0,5540.0,1482.0,638.0,844.0,2.091047e6,5539.17,0.26
"""/ESH25:XCME{=m}""",2025-03-13 20:08:00,0,7481388414617518080,0,1124,5537.25,5538.75,5537.0,5538.25,2245.0,919.0,1326.0,2.091047e6,5537.9,0.26
"""/ESH25:XCME{=m}""",2025-03-13 20:07:00,0,7481388156919480320,0,941,5539.0,5539.0,5536.75,5537.25,1534.0,941.0,593.0,2.091047e6,5538.06,0.26
"""/ESH25:XCME{=m}""",2025-03-13 20:06:00,0,7481387899221442560,0,1671,5537.0,5540.0,5536.75,5538.75,2946.0,1208.0,1738.0,2.091047e6,5538.59,0.26


In [5]:
display(Markdown(f"**Quote Feed:** {symbol}"))

display(dxlink.router.handler[Channels.Quote].processors["feed"].pl)

**Quote Feed:** /ESH25:XCME{=m}

eventSymbol,bidPrice,askPrice,bidSize,askSize
str,f64,f64,f64,f64
"""SPY""",552.59,552.63,600.0,600.0
"""SPX""",5477.19,5590.28,null,null
"""AAPL""",209.9,209.93,48.0,70.0
"""/ESH25:XCME""",5538.75,5539.0,57.0,23.0
"""BTC/USD:CXTALP""",80040.65,80602.9,0.01,1.5
"""QQQ""",469.47,469.51,200.0,150.0
"""NVDA""",115.9,115.94,4997.0,100.0


In [6]:
# TODOS

# [x] Widen the plot
# [x] Remove the scroller at the bottom
# [x] move the legent and remove Price (that is obvious)
# [x] Add MACD
# [x] Fix HULL - Align w/ candlesticks
# [x] ERROR if no study data found

# [ ] Add RSI
# [ ] Add Volume Profile (?? ... /ES, SPY, etc)
# [ ] Add velocity metric

In [ ]:
await dxlink.close()